# Tarea 1

### Ignacio Carter y Matías Garrido

Implementar la clase `OisCashflow`.

## Configuración Inicial

In [1]:
from finrisk import QC_Financial_3 as Qcf

# Modificado por AD
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

from dataclasses import dataclass
from enum import Enum
import pandas as pd
import numpy as np

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

In [4]:
#Importación y ordenamiento de datos

data  = pd.read_excel('../data/SOFR-10012019-10292020.xls', 'nice format')
df = pd.DataFrame()
df["fecha_inicial"]=pd.to_datetime(data["DATE"])
df["fecha_inicial"] = df["fecha_inicial"]
df["tasas_ois"] = data["RATE\n(PERCENT)"]
df = df.sort_values(by = "fecha_inicial")
df.index = list(range(0,len(df)))
df.style.format({'tasas_ois': '{0:.3}'})

,fecha_inicial,tasas_ois
0,2019-10-01 00:00:00,1.88
1,2019-10-02 00:00:00,1.85
2,2019-10-03 00:00:00,1.84
3,2019-10-04 00:00:00,1.82
4,2019-10-07 00:00:00,1.83
5,2019-10-08 00:00:00,1.85
6,2019-10-09 00:00:00,1.85
7,2019-10-10 00:00:00,1.85
8,2019-10-11 00:00:00,1.85
9,2019-10-15 00:00:00,2.0


## `Qcf.time_series`

In [5]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)
for d in dl:
    print(d)
print()
    
vl = Qcf.time_series_values(ts)
for v in vl:
    print(v)

ts[fecha1]: 19690113.0

13-1-1969
14-1-1969

19690113.0
19690114.0


## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

In [6]:
@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
        
    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula la tasa equivalente desde `self.start_date` a `accrual_date`. La tasa equivalente
        se calcula como:
        
        (P - 1) * 360 / (accrual_date - self.start_date)
        
        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        start_date = self.start_date
        end_date = self.end_date
        days = Qcf.time_series_dates(fixings)
        dif_day = start_date.day_diff(accrual_date) #diferencia de fechas
        factores = []
        fecha_i = start_date
        fecha_f = start_date
        P = 1
        i = 0
        
        if accrual_date > end_date:
            accrual_date = end_date
        
        while fecha_i < accrual_date and fecha_f != accrual_date:
            fecha_i = days[i]
            fecha_f = days[i+1]
            tasa = Qcf.QCInterestRate(fixings[fecha_i], Qcf.QCAct360(), Qcf.QCLinearWf())
            factor = tasa.wf(fecha_i,fecha_f)
            factores.append(factor)
            P = P*factor
            i+=1
        
        #print("Factores:", factores)
        #print("P:",P)
        
        #tasa equivalente
        tasa_equi = (P - 1)*360/dif_day 
        
        return tasa_equi
    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula el interés (plata) devengado desde `self.start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        start_date = self.start_date
        dif_day = start_date.day_diff(accrual_date) #diferencia de fechas
        nocional = self.notional
        tasa_equi = ois.get_accrued_rate(accrual_date,fixings)
        intereses = nocional*tasa_equi*dif_day/360
        
        return intereses
    
    def amount(self, fixings: Qcf.time_series) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        end_date = self.end_date
        intereses = ois.get_accrued_interest(end_date,fixings)
        amortizacion = self.amortization
        print(amortizacion)
        print(intereses)
        flujo_total = amortizacion + intereses
        
        return flujo_total
    
 #   def test(self):
     #   return 'test'
    
    #def test1(self, x: float):
        #if x == 0:
         #   raise ValueError('x debe ser != 0')
        #return 1 / x
    
   # def test2(self, x: float):
       # return self.test1(x) + 1
    
    #def test3(self, x: float):
     #   return x + self.notional

Construcción de un objeto `Qcf.InterestRateIndex`.

In [7]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

In [8]:
#Extrae del data frame fecha inicial y fecha final
day_i = df["fecha_inicial"].iloc[0].day
month_i = df["fecha_inicial"].iloc[0].month
year_i = df["fecha_inicial"].iloc[0].year

day_f = df["fecha_inicial"].iloc[-1].day
month_f = df["fecha_inicial"].iloc[-1].month
year_f = df["fecha_inicial"].iloc[-1].year

start_date = Qcf.QCDate(day_i,month_i,year_i)
end_date = Qcf.QCDate(day_f,month_f,year_f)

print("fecha inicial:", start_date)
print("fecha final:", end_date)

fecha inicial: 1-10-2019
fecha final: 28-10-2020


Construcción de una instancia de `OisCashflow`.

In [9]:
ois = OisCashflow(
    start_date,
    end_date,
    end_date,
    10000000,
    Qcf.QCUSD(),
    10000000,
    True,
    Qcf.QCInterestRate(0.1, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

In [10]:
#Tasas overnight
tasas_ois = Qcf.time_series()

for i in range(len(df)):
    dia = df["fecha_inicial"][i].day
    mes = df["fecha_inicial"][i].month
    año = df["fecha_inicial"][i].year
    fecha = Qcf.QCDate(dia,mes,año)
    tasas_ois[fecha] = df["tasas_ois"][i]/100
    
print(tasas_ois[ois.start_date])

0.018799999999999997


In [11]:
#Resultados de las 3 primeras funciones

accrual_date = Qcf.QCDate(28,10,2020)
nocional = ois.notional
tasa_equi = ois.get_accrued_rate(accrual_date,tasas_ois)
intereses = ois.get_accrued_interest(accrual_date,tasas_ois)
flujo_total = ois.amount(tasas_ois)

print("La fecha de devengo es:",accrual_date)
print(f'Tasa equivalente: {tasa_equi:.8%}')
print(f'Los intereses son de {intereses:,.2f} USD con un nocional de {nocional:,.2f}')
print(f'Flujo total: {flujo_total:,.2f} USD')

10000000
78517.17779091328
La fecha de devengo es: 28-10-2020
Tasa equivalente: 0.71924132%
Los intereses son de 78,517.18 USD con un nocional de 10,000,000.00
Flujo total: 10,078,517.18 USD


## Funciones

In [12]:
frmt = {'rate_value': '{:.8%}', 'valor_tasa': '{:.8%}',
        'nominal': '{:,.2f}', 'amortizacion': '{:,.2f}',
        'interes': '{:,.2f}', 'flujo': '{:,.2f}'
       }

In [14]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
df_curva.style.format(frmt)

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929
5,61,0.000781,0.999870
6,92,0.000811,0.999796
7,125,0.000781,0.999733
8,152,0.000760,0.999683
9,182,0.000750,0.999626


In [15]:
val_date = Qcf.QCDate(28,10,2020)
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

In [16]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en
    el notebook 9.
    """
    start_date = ois_cashflow.start_date
    end_date = ois_cashflow.end_date
    plazo = val_date.day_diff(end_date) 
    
    if val_date > end_date:
        return print("Fecha de evaluación debe ser menor o igual a la fecha de vencimiento")

    #Calculo fixings futuros
    
    for i in range(plazo):
        fecha = val_date.add_days(i)
        factor_i = zcc.get_forward_wf(0,i)
        factor_i1 = zcc.get_forward_wf(0,i+1)
        tasa = (factor_i1/factor_i - 1)*(360/1)
        fixings[fecha] = tasa
    
    tasa_esperada = ois.get_accrued_rate(end_date,fixings)
    
    return tasa_esperada


In [17]:
def present_value(val_date: Qcf.QCDate,
                  ois_cashflow: OisCashflow,
                  zcc: Qcf.ZeroCouponCurve,
                  fixings: Qcf.time_series) -> float:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Para probar esta función utilizar la data de que está en data/20201012_built_sofr_zero.xlsx.
    Para traer a valor presente debes respetar la convención de las tasas de la curva.
    
    NOTA: la variable ois_cashflow debe haber 'pasado' por la función set_expected_rate para que
    tenga los intereses de los fixings futuros.
    """
    
    tasa = set_expected_rate(val_date,ois_cashflow,zcc,fixings)
    plazo = val_date.day_diff(end_date)
    disc_fac = zcc.get_discount_factor_at(plazo)
    amortizacion = ois_cashflow.amortization
    intereses = tasa*ois_cashflow.notional
    flujo_total = amortizacion + intereses
    vp = flujo_total*disc_fac
    
    return vp

In [18]:
#Calcular valores presentes y tasa esperada

tasa_esperada = set_expected_rate(val_date,
                     ois,
                     zcc,
                     tasas_ois)

print(f'Tasa esperada: {tasa_esperada:.8%} con fecha de valoracion el {val_date}')

Tasa esperada: 0.71924132% con fecha de valoracion el 28-10-2020


In [19]:
vp = present_value(val_date,
                   ois,
                   zcc,
                   tasas_ois)
print(f'Valor presente: {vp:,.2f} USD')

Valor presente: 10,071,924.13 USD


In [20]:
#ICP(t0)*wf(t0,t2) = ICP(t2), ICP(t0)*wf(t0,t1) = ICP(t1) -> ICP(t2)/ICP(t1) = ICP(t1,t2)
#tasa(t1,t2) = [ICP(t1,t2) - 1]*360/(t2 - t1)
# ICP se puede pensar como el account de la tasa de la tasa interbancaria chilena, es el indice overnigth chileno
#Cuando  t1 < t0 < t2

#Tengo que descontar en el valor presente el flujo que obtengo con la tasa esperada calculada con el set_expected_rate
#el factor descuento lo calculo con zcc.get_disc_factor(plazo)
#la tasa para el flujo que evaluamos set_expected_rate
#flujo total = nocional(1 + tasa_expected)
#Tasa expected es mzcla entre fixings y expected deped

## Tests

In [21]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [22]:
amount_tol = 10000
rate_tol = .001

In [23]:
exitos = 0

In [24]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [25]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [26]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [27]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [28]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [29]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [30]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 1.04766527%


Se verificará el cálculo usando `df_fixings`.

In [31]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [32]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [33]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [34]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [35]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [36]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [37]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [38]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [39]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [40]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [41]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [42]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: 0.03775213%


In [43]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.accrued_interest`

In [44]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 75,082.68


In [45]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 0.000000


In [46]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.amount`

In [47]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

1000000
77850.89961860559
amount: 1,077,850.90


In [48]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [49]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: 2823.854185


In [50]:
suma_exito(amount, check_amount, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [51]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [52]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [53]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [54]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [56]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.06121277%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [57]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.06924275%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [58]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.09623647%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [59]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.09623647%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [60]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Fecha de evaluación debe ser menor o igual a la fecha de vencimiento
Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.09623647%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `present_value`

In [63]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc, fixings)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    None.day_diff(QCDate, str)
did not match C++ signature:
    day_diff(QCDate {lvalue}, QCDate)

In [64]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc, fixings)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    None.day_diff(QCDate, str)
did not match C++ signature:
    day_diff(QCDate {lvalue}, QCDate)

In [65]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc, fixings)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    None.day_diff(QCDate, str)
did not match C++ signature:
    day_diff(QCDate {lvalue}, QCDate)

In [66]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc, fixings)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    None.day_diff(QCDate, str)
did not match C++ signature:
    day_diff(QCDate {lvalue}, QCDate)

In [67]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc, fixings)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Fecha de evaluación debe ser menor o igual a la fecha de vencimiento
Fecha de evaluación debe ser menor o igual a la fecha de vencimiento


ArgumentError: Python argument types in
    None.day_diff(QCDate, str)
did not match C++ signature:
    day_diff(QCDate {lvalue}, QCDate)

In [68]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 12.0
